In [1]:
import os ,re
import time
import requests 
from requests_html import HTML 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

In [2]:
cwd = os.getcwd() # dirname of abs(path(__file__))
BASE_DIR = os.path.dirname(cwd)
BASE_DIR

'/home/vianney/projects/data-science-projects/notebooks'

In [3]:
options= Options()
options.add_argument(" --headless")
driver = webdriver.Chrome(options=options)

In [4]:
categories = [
    "https://www.amazon.com/Best-Sellers-Toys-Games/zgbs/toys-and-games/",
    "https://www.amazon.com/Best-Sellers-Electronics/zgbs/electronics/",
    "https://www.amazon.com/Best-Sellers/zgbs/fashion/"
    
]
    

In [5]:
categories

['https://www.amazon.com/Best-Sellers-Toys-Games/zgbs/toys-and-games/',
 'https://www.amazon.com/Best-Sellers-Electronics/zgbs/electronics/',
 'https://www.amazon.com/Best-Sellers/zgbs/fashion/']

In [6]:
title_lookup = "#productTitle"
price_lookup = "#a-price a-text-price a-size-medium apexPriceToPay"

In [7]:
first_url = categories[0] 

In [8]:
driver.get(first_url)

In [9]:

body_el = driver.find_element(By.CSS_SELECTOR,"body")
html_str = body_el.get_attribute("innerHTML")
html_obj = HTML(html=html_str)

In [10]:
#html_obj.links

In [11]:
#new_links = [x for x in html_obj.links if x.startswith("/")]
page_links = [f"https://www.amazon.com{x}" for x in  html_obj.links if x.startswith("/")]
#new_links = [x for x in new_links if "product_reviews/" not in x]
#print(new_links)

In [12]:
def scrape_product_page(url):
    driver.get(url)
    time.sleep(0.3)
    body_el = driver.find_element(By.CSS_SELECTOR,"body")
    html_str = body_el.get_attribute("innerHTML")
    html_obj = HTML(html=html_str)
    product_title = driver.find_element(By.ID,'productTitle').text
    product_price = driver.find_element(By.XPATH,"//span[@class='a-price a-text-price a-size-medium apexPriceToPay']").text
    return product_title , product_price

In [ ]:

#<base-url>/<slug>/dp/>product_id>/

<h3> 2- Using RegEx to parse url <h3>

In [14]:
#my_regex_pattern = r"https://www.amazon.com/(?P<slug>[\w-]+)/dp/(?P<product_id>[\w-]+)/"
#my_url = "https://www.amazon.com/First-Princess-Make-Kit-Everything/dp/B07FPWYY79/"
#regex = re.compile(my_regex_pattern)


In [15]:
#my_match = regex.match(my_url)
#print(my_match)
#my_match['product_id']

In [16]:
#my_match['slug']

In [17]:
regex_options = [
    
     r"https://www.amazon.com/gp/product/(?P<product_id>[\w-]+)/",
     r"https://www.amazon.com/dp/(?P<product_id>[\w-]+)/",
     r"https://www.amazon.com/(?P<slug>[\w-]+)/dp/(?P<product_id>[\w-]+)/"
       
]



In [18]:
new_links = [x for x in html_obj.links if x.startswith("/")]

In [19]:
def extract_product_id_from_url(url):
    product_id = None
    for regex_str in regex_options:
        regex = re.compile(regex_str)
        match = regex.match(url)
        if match != None:
            try:
                product_id = match['product_id']
            except:
                pass
    return product_id

In [20]:
#page_links = [ x for x in page_links if extract_product_id_from_url(x) != None]
def clean_page_links(page_links=[]):
    final_page_links = []
    for url in page_links:
        product_id = extract_product_id_from_url(url) 
        if product_id != None :
            final_page_links.append({"url": url, "product_id": product_id})
    return final_page_links


cleaned_links = clean_page_links(page_links)

In [21]:
len(page_links) == len(cleaned_links)

False

In [22]:
len(cleaned_links)

51

In [23]:
len(page_links)

161

In [24]:
cleaned_links 

[{'url': 'https://www.amazon.com/Tamagotchi-42881-Original-90s/dp/B08XWNK5TF/ref=zg_bs_toys-and-games_10/142-7709922-0898351?_encoding=UTF8&psc=1&refRID=32FC4FT5Z2RNRFT7J15Z',
  'product_id': 'B08XWNK5TF'},
 {'url': 'https://www.amazon.com/Family-Sturdy-Storage-Travel-Friendly-Exclusive/dp/B07P6MZPK3/ref=zg_bs_toys-and-games_34/142-7709922-0898351?_encoding=UTF8&psc=1&refRID=32FC4FT5Z2RNRFT7J15Z',
  'product_id': 'B07P6MZPK3'},
 {'url': 'https://www.amazon.com/SHASHIBO-Shifting-Geometric-Magnetic-Transforming/dp/B07W7FLKRW/ref=zg_bs_toys-and-games_1/142-7709922-0898351?_encoding=UTF8&psc=1&refRID=32FC4FT5Z2RNRFT7J15Z',
  'product_id': 'B07W7FLKRW'},
 {'url': 'https://www.amazon.com/Capsule-1-Collectible-Toy-Limited/dp/B09M4999VY/ref=zg_bs_toys-and-games_9/142-7709922-0898351?_encoding=UTF8&psc=1&refRID=32FC4FT5Z2RNRFT7J15Z',
  'product_id': 'B09M4999VY'},
 {'url': 'https://www.amazon.com/Twister-Ultimate-Colored-Compatible-Exclusive/dp/B01ALHAEWA/ref=zg_bs_toys-and-games_19/142-7709922

In [25]:
def perform_scrape(cleaned_items=[]):
    data_extracted = []
    for obj in cleaned_items:
        link= obj['url']
        product_id = obj['product_id']
        title, price = (None, None)
        try:
            title, price = scrape_product_page(link)
        except:
            pass
        if title != None and price != None:
            #print(link, title, price)
        product_data = {
            "url": link,
            "product_id": product_id,
            "title": title,
            "price": price            
            
        }
        data_extracted.append(product_data)
        
    return data_extracted

In [26]:
perform_scrape(cleaned_items=cleaned_links)

https://www.amazon.com/Tamagotchi-42881-Original-90s/dp/B08XWNK5TF/ref=zg_bs_toys-and-games_10/142-7709922-0898351?_encoding=UTF8&psc=1&refRID=32FC4FT5Z2RNRFT7J15Z Original Tamagotchi - 90s $19.99
https://www.amazon.com/Family-Sturdy-Storage-Travel-Friendly-Exclusive/dp/B07P6MZPK3/ref=zg_bs_toys-and-games_34/142-7709922-0898351?_encoding=UTF8&psc=1&refRID=32FC4FT5Z2RNRFT7J15Z UNO Family Card Game, with 112 Cards in a Sturdy Storage Tin, Travel-Friendly, Makes a Great Gift for 7 Year Olds and Up [Amazon Exclusive] $9.99
https://www.amazon.com/Twister-Ultimate-Colored-Compatible-Exclusive/dp/B01ALHAEWA/ref=zg_bs_toys-and-games_19/142-7709922-0898351?_encoding=UTF8&psc=1&refRID=32FC4FT5Z2RNRFT7J15Z Twister Ultimate: Bigger Mat, More Colored Spots, Family, Kids Party Game Age 6+; Compatible with Alexa (Amazon Exclusive) $19.99
https://www.amazon.com/Crayola-Ultimate-Drawing-Tablet-Amazon/dp/B084Y6DJ1Z/ref=zg_bs_toys-and-games_36/142-7709922-0898351?_encoding=UTF8&psc=1&refRID=32FC4FT5Z2RNR

[{'url': 'https://www.amazon.com/Tamagotchi-42881-Original-90s/dp/B08XWNK5TF/ref=zg_bs_toys-and-games_10/142-7709922-0898351?_encoding=UTF8&psc=1&refRID=32FC4FT5Z2RNRFT7J15Z',
  'product_id': 'B08XWNK5TF',
  'title': 'Original Tamagotchi - 90s',
  'price': '$19.99'},
 {'url': 'https://www.amazon.com/Family-Sturdy-Storage-Travel-Friendly-Exclusive/dp/B07P6MZPK3/ref=zg_bs_toys-and-games_34/142-7709922-0898351?_encoding=UTF8&psc=1&refRID=32FC4FT5Z2RNRFT7J15Z',
  'product_id': 'B07P6MZPK3',
  'title': 'UNO Family Card Game, with 112 Cards in a Sturdy Storage Tin, Travel-Friendly, Makes a Great Gift for 7 Year Olds and Up [Amazon Exclusive]',
  'price': '$9.99'},
 {'url': 'https://www.amazon.com/SHASHIBO-Shifting-Geometric-Magnetic-Transforming/dp/B07W7FLKRW/ref=zg_bs_toys-and-games_1/142-7709922-0898351?_encoding=UTF8&psc=1&refRID=32FC4FT5Z2RNRFT7J15Z',
  'product_id': 'B07W7FLKRW',
  'title': None,
  'price': None},
 {'url': 'https://www.amazon.com/Capsule-1-Collectible-Toy-Limited/dp/B09